# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,albany,42.6001,-73.9662,279.75,91,100,1.79,US,1702261419
1,1,port-aux-francais,-49.3500,70.2167,277.27,79,75,9.51,TF,1702261202
2,2,putina,-15.4667,-69.4333,282.36,72,96,3.90,PE,1702261420
3,3,cacimba de dentro,-6.6417,-35.7900,294.27,76,1,2.83,BR,1702261420
4,4,punta arenas,-53.1500,-70.9167,279.21,70,0,1.03,CL,1702261380


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
# Configure the map plot
city_humiditiy_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = 'Humidity',
    scale = 0.75,
    color = 'City'
)

# Display the map
city_humiditiy_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
vacation_cities_df = city_data_df[(city_data_df['Max Temp'] <= 300) & (city_data_df['Humidity'] <= 50)]

# Drop any rows with null values
vacation_cities_df = vacation_cities_df.dropna()

# Display sample data
(vacation_cities_df)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
6,6,margaret river,-33.9500,115.0667,297.47,29,0,2.54,AU,1702261422
16,16,bardsir,29.9227,56.5743,280.20,24,43,1.58,IR,1702261428
24,24,novyy turtkul',41.5500,61.0167,268.10,24,100,7.28,UZ,1702261432
35,35,salalah,17.0151,54.0924,293.20,45,0,3.60,OM,1702261439
41,41,bardai,11.6500,7.2333,289.33,29,0,4.76,NG,1702261442
...,...,...,...,...,...,...,...,...,...,...
529,529,khuzdar,27.7384,66.6434,284.45,21,2,2.90,PK,1702262006
537,537,raja,8.4596,25.6780,296.46,24,100,0.52,SS,1702261810
539,539,dakoro,14.5106,6.7650,289.53,28,1,6.99,NE,1702262012
545,545,medina,24.4686,39.6142,293.39,19,0,5.14,SA,1702261837


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = vacation_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
6,margaret river,AU,-33.9500,115.0667,29,
16,bardsir,IR,29.9227,56.5743,24,
24,novyy turtkul',UZ,41.5500,61.0167,24,
35,salalah,OM,17.0151,54.0924,45,
41,bardai,NG,11.6500,7.2333,29,
...,...,...,...,...,...,...
529,khuzdar,PK,27.7384,66.6434,21,
537,raja,SS,8.4596,25.6780,24,
539,dakoro,NE,14.5106,6.7650,28,
545,medina,SA,24.4686,39.6142,19,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": 10
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():

    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
margaret river - nearest hotel: Margaret River Hotel
bardsir - nearest hotel: No hotel found
novyy turtkul' - nearest hotel: To'rtku'l miymanxanasi
salalah - nearest hotel: Muscat International Hotel
bardai - nearest hotel: No hotel found
sao vicente - nearest hotel: Pousada Vitória
invercargill - nearest hotel: The Grand
crane - nearest hotel: No hotel found
ouarzazat - nearest hotel: Fint
papatowai - nearest hotel: No hotel found
burang - nearest hotel: No hotel found
la junta - nearest hotel: Travel Inn of La Junta
puerto deseado - nearest hotel: Los Olmos
ilorin - nearest hotel: No hotel found
bafq - nearest hotel: هتل رستوران علمدار
san patricio - nearest hotel: No hotel found
marat - nearest hotel: No hotel found
geneina - nearest hotel: Princess
erenhot - nearest hotel: No hotel found
forrest city - nearest hotel: Quality Inn
khetri - nearest hotel: Saini Hotel
al kharj - nearest hotel: شقق فندقية
ayorou - nearest hotel: Hotel Aminokal
nemuro - nearest hote

,City,Country,Lat,Lng,Humidity,Hotel Name
6,margaret river,AU,-33.9500,115.0667,29,Margaret River Hotel
16,bardsir,IR,29.9227,56.5743,24,No hotel found
24,novyy turtkul',UZ,41.5500,61.0167,24,To'rtku'l miymanxanasi
35,salalah,OM,17.0151,54.0924,45,Muscat International Hotel
41,bardai,NG,11.6500,7.2333,29,No hotel found
...,...,...,...,...,...,...
529,khuzdar,PK,27.7384,66.6434,21,Al-Mustafa hotel
537,raja,SS,8.4596,25.6780,24,No hotel found
539,dakoro,NE,14.5106,6.7650,28,No hotel found
545,medina,SA,24.4686,39.6142,19,The Oberoi Madina


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [12]:
# Configure the map plot
city_hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 1000,
    frame_height = 800,
    size = 'Humidity',
    scale = 1,
    color = 'City',
    hover_cols = ['Hotel Name', 'Country']
    )


# Display the map
city_hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)